In [1]:
DATE = "2021-06-15"
TASK = "100bp-0.1-error-rate"
DIR = "$(DATE)-$(TASK)"
DIR = mkpath("$(homedir())/$(DIR)")

"/Users/cameronprybol/2021-06-15-100bp-0.1-error-rate"

In [819]:
pkgs = [
"LightGraphs",
"MetaGraphs",
"BioSequences",
"uCSV",
"DataFrames",
"FASTX",
"HTTP",
"CodecZlib",
"DataStructures",
"Revise",
"ProgressMeter",
"BenchmarkTools",
"Random",
"StatsBase",
"SparseArrays",
"Statistics",
"BioAlignments",
"NumericIO"
]

import Pkg
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $(basename(pkg))"))
end

import Mycelia

In [ ]:
# # 11, 17 is too small
# for k in [13, 17, 23, 31, 41, 53]
#     @show k
#     outgraph = "$(DIR)/simplified-kayvirus.k_$(k).all_ncbi.gfa"
#     # if !isfile(outgraph)
#     accessions = ncbi_staph_phage_metadata[!, "Accession"]
#     fastx_iterator = (Mycelia.get_sequence(db="nuccore", accession=accession) for accession in accessions)
#     KMER_TYPE = BioSequences.BigDNAMer{k}
#     @time kmer_graph = Mycelia.fastx_to_kmer_graph(KMER_TYPE, fastx_iterator)

#     @info "simplifying kmer graph"
#     @info "resolving untigs..."
#     @time untigs = Mycelia.resolve_untigs(kmer_graph)

#     @info "determining untig orientations..."
#     @time oriented_untigs = Mycelia.determine_oriented_untigs(kmer_graph, untigs)

#     simplified_graph = MetaGraphs.MetaDiGraph(length(oriented_untigs))
#     MetaGraphs.set_prop!(simplified_graph, :k, kmer_graph.gprops[:k])
#     @info "initializing graph node metadata"
#     ProgressMeter.@showprogress for (vertex, untig) in enumerate(oriented_untigs)
#         MetaGraphs.set_prop!(simplified_graph, vertex, :sequence, untig.sequence)
#         MetaGraphs.set_prop!(simplified_graph, vertex, :path, untig.path)
#         MetaGraphs.set_prop!(simplified_graph, vertex, :orientations, untig.orientations)
#         MetaGraphs.set_prop!(simplified_graph, vertex, :evidence, untig.evidence)
#     end

#     # determine oriented edges of simplified graph
#     simplified_untigs = Vector{Pair{Pair{Int64,Bool},Pair{Int64,Bool}}}(undef, length(LightGraphs.vertices(simplified_graph)))
#     @info "creating simplified unitgs to help resolve connections"
#     # use a pre-allocated array here to speed up
#     ProgressMeter.@showprogress for vertex in LightGraphs.vertices(simplified_graph)
#         in_kmer = simplified_graph.vprops[vertex][:path][1] => simplified_graph.vprops[vertex][:orientations][1]
#         out_kmer = simplified_graph.vprops[vertex][:path][end] => simplified_graph.vprops[vertex][:orientations][end]
#     #     @show vertex, in_kmer, out_kmer
#         simplified_untigs[vertex] = in_kmer => out_kmer
#     #     push!(simplified_untigs, )
#     end

#     # make a dictionary mapping endcap to oriented_untig index

#     end_mer_map = Dict()
#     ProgressMeter.@showprogress for (i, oriented_untig) in enumerate(oriented_untigs)
#         end_mer_map[first(oriented_untig.path)] = i
#         end_mer_map[last(oriented_untig.path)] = i
#     end

#     ProgressMeter.@showprogress for (untig_index, oriented_untig) in enumerate(oriented_untigs)
#     #     @show untig_index
#         true_in_overlap = oriented_untig.sequence[1:simplified_graph.gprops[:k]-1]

#         non_backtracking_neighbors = LightGraphs.neighbors(kmer_graph, oriented_untig.path[1])
#         if length(oriented_untig.path) > 1
#             non_backtracking_neighbors = setdiff(non_backtracking_neighbors, oriented_untig.path[2])
#         end
#         for non_backtracking_neighbor in non_backtracking_neighbors
#             neighboring_untig_index = end_mer_map[non_backtracking_neighbor]
#             neighboring_untig = oriented_untigs[neighboring_untig_index]

#             neighbor_true_out_overlap = neighboring_untig.sequence[end-simplified_graph.gprops[:k]+2:end]
#             if neighbor_true_out_overlap == true_in_overlap
#                 e = LightGraphs.Edge(neighboring_untig_index, untig_index)
#     #             o = true => true
#                 o = (source_orientation = true, destination_orientation = true)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end

#             neighbor_false_out_overlap = BioSequences.reverse_complement(neighboring_untig.sequence)[end-simplified_graph.gprops[:k]+2:end]
#             if neighbor_false_out_overlap == true_in_overlap        
#                 e = LightGraphs.Edge(neighboring_untig_index, untig_index)
#     #             o = false => true
#                 o = (source_orientation = false, destination_orientation = true)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end
#         end

#         true_out_overlap = oriented_untig.sequence[end-simplified_graph.gprops[:k]+2:end]
#         non_backtracking_neighbors = LightGraphs.neighbors(kmer_graph, oriented_untig.path[end])
#         if length(oriented_untig.path) > 1
#             non_backtracking_neighbors = setdiff(non_backtracking_neighbors, oriented_untig.path[end-1])
#         end
#         for non_backtracking_neighbor in non_backtracking_neighbors
#             neighboring_untig_index = end_mer_map[non_backtracking_neighbor]
#             neighboring_untig = oriented_untigs[neighboring_untig_index]

#             neighbor_true_in_overlap = neighboring_untig.sequence[1:simplified_graph.gprops[:k]-1]
#             if true_out_overlap == neighbor_true_in_overlap
#                 e = LightGraphs.Edge(untig_index, neighboring_untig_index)
#     #             o = true => true
#                 o = (source_orientation = true, destination_orientation = true)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end

#             neighbor_false_in_overlap = BioSequences.reverse_complement(neighboring_untig.sequence)[1:simplified_graph.gprops[:k]-1]
#             if true_out_overlap == neighbor_false_in_overlap
#                 e = LightGraphs.Edge(untig_index, neighboring_untig_index)
#     #             o = true => false
#                 o = (source_orientation = true, destination_orientation = false)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end
#         end

#         false_in_overlap = BioSequences.reverse_complement(oriented_untig.sequence)[1:simplified_graph.gprops[:k]-1]

#         non_backtracking_neighbors = LightGraphs.neighbors(kmer_graph, oriented_untig.path[end])
#         if length(oriented_untig.path) > 1
#             non_backtracking_neighbors = setdiff(non_backtracking_neighbors, oriented_untig.path[end-1])
#         end
#         for non_backtracking_neighbor in non_backtracking_neighbors
#             neighboring_untig_index = end_mer_map[non_backtracking_neighbor]
#             neighboring_untig = oriented_untigs[neighboring_untig_index]

#             neighbor_true_out_overlap = neighboring_untig.sequence[end-simplified_graph.gprops[:k]+2:end]
#             if neighbor_true_out_overlap == false_in_overlap
#                 e = LightGraphs.Edge(neighboring_untig_index, untig_index)
#     #             o = true => false
#                 o = (source_orientation = true, destination_orientation = false)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end

#             neighbor_false_out_overlap = BioSequences.reverse_complement(neighboring_untig.sequence)[end-simplified_graph.gprops[:k]+2:end]
#             if neighbor_false_out_overlap == false_in_overlap        
#                 e = LightGraphs.Edge(neighboring_untig_index, untig_index)
#     #             o = false => false
#                 o = (source_orientation = false, destination_orientation = false)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end
#         end

#         false_out_overlap = BioSequences.reverse_complement(oriented_untig.sequence)[end-simplified_graph.gprops[:k]+2:end]

#         non_backtracking_neighbors = LightGraphs.neighbors(kmer_graph, oriented_untig.path[1])
#         if length(oriented_untig.path) > 1
#             non_backtracking_neighbors = setdiff(non_backtracking_neighbors, oriented_untig.path[2])
#         end

#         for non_backtracking_neighbor in non_backtracking_neighbors
#             neighboring_untig_index = end_mer_map[non_backtracking_neighbor]
#             neighboring_untig = oriented_untigs[neighboring_untig_index]

#             neighbor_true_in_overlap = neighboring_untig.sequence[1:simplified_graph.gprops[:k]-1]
#             if false_out_overlap == neighbor_true_in_overlap
#                 e = LightGraphs.Edge(untig_index, neighboring_untig_index)
#     #             o = false => true
#                 o = (source_orientation = false, destination_orientation = true)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end

#             neighbor_false_in_overlap = BioSequences.reverse_complement(neighboring_untig.sequence)[1:simplified_graph.gprops[:k]-1]
#             if false_out_overlap == neighbor_false_in_overlap
#                 e = LightGraphs.Edge(untig_index, neighboring_untig_index)
#     #             o = false => false
#                 o = (source_orientation = false, destination_orientation = false)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end
#         end
#     end

#     # ^ > 1 day down to 34 seconds!
#     Mycelia.graph_to_gfa(simplified_graph, outgraph)
# end

In [3]:
# set a random seed
seed = Random.seed!(0)

MersenneTwister(0)

In [4]:
# randomly generate a dna sequence of 100bp
genome = BioSequences.randdnaseq(seed, 100)

100nt DNA Sequence:
AAGGGTGCGGTCTAGGTGCACTGCTTATGGTCCCCGACA…TCTACCTCTTTGTTGAAACGTATTCTTCTCTTAACACCT

In [5]:
# define error rate
error_rate = 0.1

0.1

In [6]:
# generate 100x coverage fastq file
# put accuracy rate into fastq file
coverage = 100
fastq_file = "$(DIR)/$(DATE)-$(TASK).fastq"
open(fastq_file, "w") do io
    fastq_writer = FASTX.FASTQ.Writer(io)
    for i in 1:coverage
        observed_sequence = Mycelia.observe(genome, error_rate=error_rate)
        q = -10 * log10(error_rate)
        quality_scores = fill(q, length(observed_sequence))
        fastq_record = FASTX.FASTQ.Record("i", observed_sequence, quality_scores)
        write(fastq_writer, fastq_record)
    end
end

In [7]:
# build weighted kmer graph
# at some point where we may want to record weights as integers rather than lists of evidence, but first confirm the algorithm

In [8]:
# k = Int(1/error_rate + 1)

11

In [550]:
k = 5

5

In [551]:
kmer_type = BioSequences.BigDNAMer{k}

BioSequences.BigDNAMer{5} (alias for BioSequences.BigMer{BioSequences.DNAAlphabet{2}, 5})

In [552]:
kmer_graph = Mycelia.fastx_to_kmer_graph(kmer_type, fastq_file)

┌ Info: assessing kmers
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:1877
┌ Info: creating graph
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:1891


{501, 2900} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [553]:
# visualize
gfa_file = fastq_file * ".k-$k.gfa"
Mycelia.graph_to_gfa(kmer_graph, gfa_file)

"/Users/cameronprybol/2021-06-15-100bp-0.1-error-rate/2021-06-15-100bp-0.1-error-rate.fastq.k-5.gfa"

In [554]:
run(`/Applications/Bandage.app/Contents/MacOS/Bandage image $(gfa_file) $(gfa_file).svg --depwidth 1 --deppower 1`)
# --nodewidth <float> Average node width (0.5 to 1000, default: 5)
# --depwidth <float>  Depth effect on width (0 to 1, default: 0.5)
# --deppower <float>  Power of depth effect on width (0 to 1, default: 0.5)

Process(`/Applications/Bandage.app/Contents/MacOS/Bandage image /Users/cameronprybol/2021-06-15-100bp-0.1-error-rate/2021-06-15-100bp-0.1-error-rate.fastq.k-5.gfa /Users/cameronprybol/2021-06-15-100bp-0.1-error-rate/2021-06-15-100bp-0.1-error-rate.fastq.k-5.gfa.svg --depwidth 1 --deppower 1`, ProcessExited(0))

In [555]:
html_path_to_svg = "./" * repeat("../", length(split(pwd(), '/')) - 3)
html_path_to_svg *= replace("$(gfa_file).svg", "$(homedir())/" => "")

"./../../../../2021-06-15-100bp-0.1-error-rate/2021-06-15-100bp-0.1-error-rate.fastq.k-5.gfa.svg"

In [556]:
x = "<img src=$(html_path_to_svg)>"
display("text/html", x)
# display("image/svg+xml", read("$(gfa_file).svg", String))

In [557]:
fastq_io = FASTX.FASTQ.Reader(open(fastq_file))

FASTX.FASTQ.Reader{TranscodingStreams.NoopStream{IOStream}}(BioGenerics.Automa.State{TranscodingStreams.NoopStream{IOStream}}(TranscodingStreams.NoopStream{IOStream}(<mode=idle>), 1, 1, false), nothing)

In [558]:
fastq_record = first(fastq_io)

FASTX.FASTQ.Record:
   identifier: i
  description: <missing>
     sequence: AGATGTTAAGAGAGAGAATACTTTTCTAACAAAGTAGGTAATCAGATAATCACGCACGGTCCTGTCGAGGGACCCATAAGCAGTTCGCCAGACCGCACCCTT
      quality: UInt8[0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a  …  0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a]

In [559]:
# for q in FASTX.quality(fastq_read)
#     q = -10 * log10(error_rate)
#     likelihood_error = 10^(q/(-10))
#     display(likelihood_error)
# end

In [560]:
kmers = [kmer_graph.vprops[v][:kmer] for v in LightGraphs.vertices(kmer_graph)]

501-element Vector{BioSequences.BigDNAMer{5}}:
 AAAAA
 AAAAC
 AAAAG
 AAAAT
 AAACA
 AAACC
 AAACG
 AAACT
 AAAGA
 AAAGC
 AAAGG
 AAAGT
 AAATA
 ⋮
 TCGAA
 TCGCA
 TCTAA
 TCTCA
 TGAAA
 TGACA
 TGCAA
 TGCCA
 TGGAA
 TGTAA
 TTAAA
 TTCAA

In [561]:
kmer_counts = [length(kmer_graph.vprops[v][:evidence]) for v in LightGraphs.vertices(kmer_graph)]

501-element Vector{Int64}:
 10
  3
 10
  8
 12
  6
 24
  4
 23
  8
 11
  5
  5
  ⋮
  5
  3
 10
 10
 17
  6
 10
  1
  1
  3
 15
 20

In [562]:
StatsBase.describe(kmer_counts)

Summary Stats:
Length:         501
Missing Count:  0
Mean:           9.103792
Minimum:        1.000000
1st Quartile:   4.000000
Median:         7.000000
3rd Quartile:   11.000000
Maximum:        41.000000
Type:           Int64


In [563]:
kmer_likelihoods = kmer_counts ./ sum(kmer_counts)

501-element Vector{Float64}:
 0.0021925016443762333
 0.00065775049331287
 0.0021925016443762333
 0.0017540013155009867
 0.00263100197325148
 0.00131550098662574
 0.00526200394650296
 0.0008770006577504934
 0.005042753782065337
 0.0017540013155009867
 0.0024117518088138568
 0.0010962508221881166
 0.0010962508221881166
 ⋮
 0.0010962508221881166
 0.00065775049331287
 0.0021925016443762333
 0.0021925016443762333
 0.0037272527954395965
 0.00131550098662574
 0.0021925016443762333
 0.00021925016443762334
 0.00021925016443762334
 0.00065775049331287
 0.00328875246656435
 0.0043850032887524665

In [564]:
StatsBase.describe(kmer_likelihoods)

Summary Stats:
Length:         501
Missing Count:  0
Mean:           0.001996
Minimum:        0.000219
1st Quartile:   0.000877
Median:         0.001535
3rd Quartile:   0.002412
Maximum:        0.008989
Type:           Float64


In [565]:
# evaluate likelihood of first read

In [566]:
kmer_type = BioSequences.BigDNAMer{kmer_graph.gprops[:k]}

BioSequences.BigDNAMer{5} (alias for BioSequences.BigMer{BioSequences.DNAAlphabet{2}, 5})

In [567]:
K = length(LightGraphs.vertices(kmer_graph))

501

In [568]:
transition_likelihoods = Dict(
    true => SparseArrays.spzeros(K, K),
    false => SparseArrays.spzeros(K, K)
)

for edge in collect(LightGraphs.edges(kmer_graph))
    weight = length(kmer_graph.eprops[edge][:evidence])
    for o in kmer_graph.eprops[edge][:orientations]
        transition_likelihoods[o.source_orientation][edge.src, edge.dst] = weight
    end
end

for source_orientation in (true, false)
    for src in 1:K
        transition_weights = transition_likelihoods[source_orientation][src, :]
        total_weight = sum(transition_weights)
        dsts, vals = SparseArrays.findnz(transition_weights)
        for (dst, val) in zip(dsts, vals) 
            transition_likelihoods[source_orientation][src, dst] = val / total_weight
        end
        normalized_probability = sum(transition_likelihoods[source_orientation][src, :])
        @assert isapprox(normalized_probability, 0) || isapprox(normalized_probability, 1)
    end
end

In [569]:
# evaluate probability of each state

In [570]:
number_of_kmers_in_read = length(FASTX.sequence(fastq_record))-k+1

98

In [571]:
# maybe don't pre-calculate the shortest paths?
# shortest_paths = LightGraphs.enumerate_paths(LightGraphs.floyd_warshall_shortest_paths(kmer_graph))

In [572]:
# src = first(LightGraphs.edges(kmer_graph)).src
# dst = first(LightGraphs.edges(kmer_graph)).dst
# LightGraphs.a_star(kmer_graph, src, dst)

# Initialize

In [755]:
state_likelihoods = zeros(K, number_of_kmers_in_read)

501×98 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

In [756]:
arrival_paths = fill(Pair{Int, Bool}[], K, number_of_kmers_in_read)

501×98 Matrix{Vector{Pair{Int64, Bool}}}:
 []  []  []  []  []  []  []  []  []  …  []  []  []  []  []  []  []  []  []
 []  []  []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 []  []  []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 []  []  []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 []  []  []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 []  []  []  []  []  []  []  []  []  …  []  []  []  []  []  []  []  []  []
 []  []  []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 []  []  []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 []  []  []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 []  []  []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 []  []  []  []  []  []  []  []  []  …  []  []  []  []  []  []  []  []  []
 []  []  []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 []  []  []  []  []  []  []  []  []     []  []  []  []  []

# Determine Likelihoods of initial states

In [757]:
initial_state = first(BioSequences.each(kmer_type, FASTX.sequence(fastq_record)))

Mer iteration result:
Position: 1
Forward: AGATG
Backward: CATCT


In [758]:
current_state = 1

1

In [759]:
total_states = length(FASTX.sequence(fastq_record)) - k + 1

98

In [760]:
# sequence_kmer_iterator

In [761]:
# for state in sequence_kmer_iterator[2:end]
#     @show state
# end

In [762]:
# note this is a place for potential improvement, use the q value at each base to guide probability rather than median
median_q_value = Statistics.median(Int.(FASTX.quality(fastq_record)[1:k]))

10.0

In [763]:
function q_value_to_error_rate(q_value)
    error_rate = 10^(q_value/(-10))
    return error_rate
end

function error_rate_to_q_value(error_rate)
    q_value = -10 * log10(error_rate)
    return q_value
end

error_rate_to_q_value (generic function with 1 method)

In [764]:
current_error_rate = q_value_to_error_rate(median_q_value)

0.1

In [765]:
canonical_kmer = BioSequences.canonical(initial_state.fw)

DNA 5-mer:
AGATG

In [766]:
observed_orientation = initial_state.fw == initial_canonical_kmer

false

In [767]:
x = nothing
for vertex in collect(LightGraphs.vertices(kmer_graph))
    hidden_kmer = kmer_graph.vprops[vertex][:kmer]

    fw_alignment = 
        BioAlignments.pairalign(
            BioAlignments.LevenshteinDistance(), 
            canonical_kmer, 
            hidden_kmer)
    
    fw_probability = kmer_likelihoods[vertex]
    
    for match in 1:BioAlignments.count_matches(BioAlignments.alignment(fw_alignment))
        fw_probability *= 1 - error_rate
    end

    for edit in 1:fw_alignment.value
        fw_probability *= error_rate
    end
    
    bw_alignment = 
        BioAlignments.pairalign(
            BioAlignments.LevenshteinDistance(),
            BioSequences.reverse_complement(canonical_kmer),
            hidden_kmer)
    
    bw_probability = kmer_likelihoods[vertex]
    
    for match in 1:BioAlignments.count_matches(BioAlignments.alignment(bw_alignment))
        bw_probability *= 1 - error_rate
    end

    for edit in 1:bw_alignment.value
        bw_probability *= error_rate
    end
    
    if fw_probability >= bw_probability
        state_probability = fw_probability
        state_orientation = true
    else
        state_probability = bw_probability
        state_orientation = false
    end
    state_likelihoods[vertex, current_state] = state_probability
    arrival_paths[vertex, current_state] = [vertex => state_orientation]
end

In [768]:
state_likelihoods

501×98 Matrix{Float64}:
 1.77593e-6  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 5.32778e-7  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.59833e-5  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.42074e-6  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 2.13111e-6  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.06556e-6  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 3.836e-5    0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 6.39333e-6  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 4.08463e-6  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.42074e-6  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.75817e-5  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 8.87963e-7  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 7.99167e-6  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 

In [769]:
arrival_paths

501×98 Matrix{Vector{Pair{Int64, Bool}}}:
 [1=>1]    []  []  []  []  []  []  []  …  []  []  []  []  []  []  []  []  []
 [2=>1]    []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 [3=>1]    []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 [4=>1]    []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 [5=>1]    []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 [6=>1]    []  []  []  []  []  []  []  …  []  []  []  []  []  []  []  []  []
 [7=>1]    []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 [8=>0]    []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 [9=>1]    []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 [10=>1]   []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 [11=>1]   []  []  []  []  []  []  []  …  []  []  []  []  []  []  []  []  []
 [12=>1]   []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 [13=>1]   []  []  []  []  []  [] 

In [770]:
@time shortest_paths = LightGraphs.enumerate_paths(LightGraphs.floyd_warshall_shortest_paths(kmer_graph));

  0.263547 seconds (956.76 k allocations: 85.608 MiB, 15.86% gc time)


# Determine Likelihood of future states

In [771]:
# need to get just the next base, don't need to worry about the kmers anymore

In [772]:
# x = rand(10)

In [773]:
# sortperm(x)

In [774]:
# ?sortperm!()

In [775]:
kmer_graph.eprops[LightGraphs.Edge(429, 395)][:orientations]

Set{NamedTuple{(:source_orientation, :destination_orientation), Tuple{Bool, Bool}}} with 1 element:
  (source_orientation = false, destination_orientation = true)

In [779]:
function run_viterbi!(
        current_state,
        prior_state,
        observed_nucleotide,
        observed_quality_score,
        observed_error_rate,
        current_vertex,
        prior_vertex,
        state_likelihoods,
        shortest_paths,
        arrival_paths,
        kmer_graph,
        kmer_likelihoods
        )
    # if probability of prior state is lower than current probability, skip

    current_state_likelihood = state_likelihoods[current_vertex, current_state]

    prior_state_likelihood = state_likelihoods[prior_vertex, prior_state]

    # if we already have a better possible path, skip calculating anything
    if prior_state_likelihood < current_state_likelihood
        return
    end

#         # if current state is unreachable from prior state, skip
#         if !LightGraphs.has_path(kmer_graph, prior_vertex, current_vertex)
#             continue
#     #         @info "no path, skip me"
#         end

#         shortest_path = LightGraphs.a_star(kmer_graph, prior_vertex, current_vertex)
    shortest_path = shortest_paths[prior_vertex][current_vertex]

#     no path & not considering insertion
    if isempty(shortest_path) && (prior_vertex != current_vertex)
        @info "can I even get here?"
        return
    end

    # single step - either match or mismatch

    # multistep path - deletion in observed sequence relative to kmer graph

    # one or more step path - match, mismatch, or deletion in observed sequence relative to kmer graph
    # (vertex, orientation) = last(arrival_paths[prior_vertex, prior_state])
    if !isempty(shortest_path)

        initial_path_state = last(arrival_paths[prior_vertex, prior_state])

        path = Vector{typeof(initial_path_state)}(undef, length(shortest_path))
        path[1] = initial_path_state

        path_likelihood::Float64 = state_likelihoods[prior_vertex, prior_state]

#             for (i, vertex) in zip(2:length(shortestshortest_path)
        for i in 2:length(shortest_path)

            this_vertex = shortest_path[i]
            prior_vertex, prior_orientation = path[i-1]
            edge = LightGraphs.Edge(prior_vertex, this_vertex)

            possible_edge_orientations::Set{NamedTuple{(:source_orientation, :destination_orientation), Tuple{Bool, Bool}}} = kmer_graph.eprops[edge][:orientations]
            possible_edge_orientations = filter(o -> o.source_orientation == prior_orientation, possible_edge_orientations)

            if isempty(possible_edge_orientations)
                path_likelihood *= 0.0
                path = Vector{eltype(path)}()
                break
            end

#             NamedTuple{(:source_orientation, :destination_orientation), Tuple{Bool, Bool}}
            orientation = first(possible_edge_orientations)
            # transition likelihood * state_likelihood
            transition_likelihood::Float64 = transition_likelihoods[prior_orientation][prior_vertex, this_vertex]
            state_likelihood::Float64 = kmer_likelihoods[this_vertex]
            path_likelihood *= transition_likelihood * state_likelihood
            path[i] = this_vertex => orientation.destination_orientation
        end

        # see if new nucleotide is a match or mismatch to terminal kmer in path
        if !isempty(path) && path_likelihood > 0
            terminal_kmer_index, terminal_kmer_orientation = last(path)
            terminal_kmer = BioSequences.LongDNASeq(kmer_graph.vprops[terminal_kmer_index][:kmer])::BioSequences.LongDNASeq
            if !terminal_kmer_orientation
                terminal_kmer = BioSequences.reverse_complement!(terminal_kmer)
            end
            is_match = observed_nucleotide == last(terminal_kmer)
            if is_match
                path_likelihood *= 1 - observed_error_rate
            else
                path_likelihood *= observed_error_rate
            end
        end

        if path_likelihood > state_likelihoods[current_vertex, current_state]
            state_likelihoods[current_vertex, current_state] = path_likelihood
            arrival_paths[current_vertex, current_state] = path
        end
    # zero step path - insertion in observed sequence relative to kmer graph
    elseif current_vertex == prior_vertex
        emission_likelihood = observed_error_rate
        transition_likelihood = observed_error_rate
        state_likelihood = kmer_likelihoods[current_vertex]
        current_state_likelihood = prior_state_likelihood * emission_likelihood * transition_likelihood * state_likelihood

        if current_state_likelihood > state_likelihoods[current_vertex, state_index]
            state_likelihoods[current_vertex, state_index] = current_state_likelihood
            arrival_paths[current_vertex, state_index] = arrival_paths[prior_vertex, state_index-1]
        end
    end
    return
end

run_viterbi! (generic function with 2 methods)

In [ ]:
current_vertex = 1
prior_vertex = 1
@code_warntype run_viterbi!(
    current_state,
    prior_state,
    observed_nucleotide,
    observed_quality_score,
    observed_error_rate,
    current_vertex,
    prior_vertex,
    state_likelihoods,
    shortest_paths,
    arrival_paths,
    kmer_graph,
    kmer_likelihoods
    )

In [781]:
# consider each state
ProgressMeter.@showprogress for current_state in 2:total_states
    prior_state = current_state - 1
    observed_nucleotide = FASTX.sequence(fastq_record)[k-1+current_state]
    observed_quality_score = FASTX.quality(fastq_record)[k-1+current_state]
    observed_error_rate = q_value_to_error_rate(observed_quality_score)

    for current_vertex in 1:K
        for prior_vertex in 1:K
            run_viterbi!(
                current_state,
                prior_state,
                observed_nucleotide,
                observed_quality_score,
                observed_error_rate,
                current_vertex,
                prior_vertex,
                state_likelihoods,
                shortest_paths,
                arrival_paths,
                kmer_graph,
                kmer_likelihoods
                )
        end
    end
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:55


In [782]:
# if 
state_likelihoods

501×98 Matrix{Float64}:
 1.77593e-6  3.89372e-11  6.22347e-13  …  2.91962e-278  1.98209e-281
 5.32778e-7  9.46174e-10  5.09193e-13     3.2142e-281   8.17197e-282
 1.59833e-5  3.50435e-10  1.8859e-13      3.56354e-279  3.20718e-279
 1.42074e-6  4.67247e-10  6.03197e-12     4.01791e-278  1.13713e-281
 2.13111e-6  8.41044e-10  5.84465e-12     7.43917e-278  5.25634e-281
 1.06556e-6  5.16095e-9   2.15747e-13  …  2.85136e-280  2.56623e-280
 3.836e-5    1.08134e-8   7.59198e-13     4.14137e-277  3.72724e-277
 6.39333e-6  1.40174e-10  1.84399e-13     2.1094e-279   8.60972e-286
 4.08463e-6  7.254e-9     2.65074e-13     1.88317e-276  5.64644e-279
 1.42074e-6  1.12139e-9   2.47449e-14     2.92002e-278  4.29028e-281
 1.75817e-5  9.63696e-10  1.01419e-13  …  7.29269e-276  6.56342e-276
 8.87963e-7  1.57696e-9   1.29656e-13     8.12665e-276  9.08795e-282
 7.99167e-6  4.86715e-10  3.88967e-12     1.98368e-281  1.04351e-282
 ⋮                                     ⋱                
 8.87963e-8  5.84058e-

In [783]:
arrival_paths

501×98 Matrix{Vector{Pair{Int64, Bool}}}:
 [1=>1]    [1=>1]            …  [4=>0, 1=>0]
 [2=>1]    [8=>0, 2=>0]         [7=>0, 2=>0]
 [3=>1]    [3=>1]               [12=>0, 3=>0]
 [4=>1]    [1=>1, 4=>1]         [14=>0, 4=>0]
 [5=>1]    [20=>0, 5=>0]        [72=>0, 19=>0, 5=>0]
 [6=>1]    [24=>0, 6=>0]     …  [87=>0, 23=>0, 6=>0]
 [7=>1]    [28=>0, 7=>0]        [28=>0, 7=>0]
 [8=>0]    [2=>1, 8=>1]         [105=>0, 29=>0, 8=>0]
 [9=>1]    [3=>1, 9=>1]         [34=>0, 9=>0]
 [10=>1]   [36=>0, 10=>0]       [38=>0, 10=>0]
 [11=>1]   [43=>0, 11=>0]    …  [43=>0, 11=>0]
 [12=>1]   [3=>1, 12=>1]        [44=>0, 12=>0]
 [13=>1]   [48=>0, 13=>0]       [47=>0, 13=>0]
 ⋮                           ⋱  
 [490=>1]  [214=>1, 490=>0]     [214=>1, 490=>0]
 [491=>0]  [491=>0]          …  [241=>0, 342=>1, 319=>0, 491=>0]
 [492=>0]  [346=>0, 492=>0]     [112=>1, 359=>1, 492=>1]
 [493=>0]  [199=>1, 493=>0]     [393=>0, 493=>1]
 [494=>1]  [219=>1, 494=>0]     [11=>0, 376=>1, 494=>0]
 [495=>1]  [207=>1, 495=>0]

In [784]:
# take shortest path and assume it's the maximum likelihood path
# this assumption seems fair because in an ideal situation
# we're just moving to an adjacent kmer
# and the shortest path and most likely path should be the same

In [785]:


# for very distant nodes
# it's very unlikely that any path between them will be correct
# so taking shortest or the most likely won't matter since a different, shorter path will be more likely

In [ ]:
# backtrack most likely path

In [786]:
maximum_likelihood_path_value = maximum(state_likelihoods[:, end])

9.87037849640755e-274

In [787]:
maximum_likelihood_path_indices = findall(state_likelihoods[:, end] .== maximum_likelihood_path_value)

1-element Vector{Int64}:
 42

In [788]:
# if multiple paths are tied, randomly choose one
maximum_likelihood_path_index = rand(maximum_likelihood_path_indices)

42

In [789]:
maximum_likelihood_path = arrival_paths[maximum_likelihood_path_index, end]

2-element Vector{Pair{Int64, Bool}}:
 83 => 1
 42 => 0

In [790]:
for index in total_states-1:-1:1
#     @show index
    prior_state, prior_orientation = first(maximum_likelihood_path)
    maximum_likelihood_arrival_path = arrival_paths[prior_state, index]
    @assert last(maximum_likelihood_arrival_path) == (prior_state => prior_orientation)
    maximum_likelihood_path = vcat(maximum_likelihood_arrival_path[1:end-1], maximum_likelihood_path)
end
maximum_likelihood_path

98-element Vector{Pair{Int64, Bool}}:
 158 => 1
  66 => 0
  18 => 0
 473 => 0
 468 => 1
 369 => 0
 474 => 1
  34 => 1
 127 => 1
 360 => 0
 127 => 1
 360 => 0
 127 => 1
     ⋮
 349 => 0
 457 => 1
 463 => 0
 124 => 1
 335 => 0
  85 => 1
 291 => 1
 319 => 1
 410 => 1
 237 => 1
  83 => 1
  42 => 0

In [791]:
function oriented_path_to_sequence(kmer_graph, oriented_path)
    initial_kmer, initial_orientation = first(oriented_path)
    k = kmer_graph.gprops[:k]
    sequence = BioSequences.LongDNASeq(kmer_graph.vprops[initial_kmer][:kmer])
    if !initial_orientation
        sequence = BioSequences.reverse_complement(sequence)
    end
    for (kmer, orientation) in oriented_path[2:end]
        kmer_seq = BioSequences.LongDNASeq(kmer_graph.vprops[kmer][:kmer])
        if !orientation
            kmer_seq = BioSequences.reverse_complement(kmer_seq)
        end
        @assert sequence[end-k+2:end] == kmer_seq[1:end-1]
        push!(sequence, last(kmer_seq))
    end
    return sequence
end

oriented_path_to_sequence (generic function with 1 method)

In [795]:
maximum_likelihood_sequence = oriented_path_to_sequence(kmer_graph, maximum_likelihood_path)

102nt DNA Sequence:
AGGTGTTAAGAGAGAGAATACGTTTCAAAAGAGGTAGAT…GTCGGGGGACCCATAAGCAGTGCGGTAGACCGCACCCTT

102nt DNA Sequence:
AGATGTTAAGAGAGAGAATACTTTTCTAACAAAGTAGGT…GTCGAGGGACCCATAAGCAGTTCGCCAGACCGCACCCTT

In [796]:
original_sequence_likelihood = 1.0

1.0

In [797]:
function sequence_to_oriented_path(sequence, kmers::Vector{T}) where {T <: BioSequences.AbstractMer{A, K}} where {A, K}
    observed_path = Vector{Pair{Int, Bool}}(undef, length(sequence)-K+1)
    for (i, kmer) in enumerate(BioSequences.each(T, sequence))
        canonical_kmer = BioSequences.canonical(kmer.fw)
        index = Mycelia.get_kmer_index(kmers, canonical_kmer)
        orientation = kmer.fw == canonical_kmer
        observed_path[i] = index => orientation
    end
    return observed_path
end

sequence_to_oriented_path (generic function with 1 method)

98-element Vector{Pair{Int64, Bool}}:
 132 => 1
  74 => 0
  20 => 0
 473 => 0
 468 => 1
 369 => 0
 474 => 1
  34 => 1
 127 => 1
 360 => 0
 127 => 1
 360 => 0
 127 => 1
     ⋮
 366 => 0
 273 => 1
 246 => 1
 124 => 1
 335 => 0
  85 => 1
 291 => 1
 319 => 1
 410 => 1
 237 => 1
  83 => 1
  42 => 0

In [801]:
# function oriented_path_to_likelihood(kmer_graph, kmer_likelihoods, transition_likelihoods, fastq_record)
original_oriented_path = sequence_to_oriented_path(FASTX.sequence(fastq_record), kmers)

98-element Vector{Pair{Int64, Bool}}:
 132 => 1
  74 => 0
  20 => 0
 473 => 0
 468 => 1
 369 => 0
 474 => 1
  34 => 1
 127 => 1
 360 => 0
 127 => 1
 360 => 0
 127 => 1
     ⋮
 366 => 0
 273 => 1
 246 => 1
 124 => 1
 335 => 0
  85 => 1
 291 => 1
 319 => 1
 410 => 1
 237 => 1
  83 => 1
  42 => 0

In [810]:
likelihood = kmer_likelihoods[first(first(original_oriented_path))]

0.0024117518088138568

In [811]:
for q_value in FASTX.quality(fastq_record)[1:kmer_graph.gprops[:k]]
    error_rate = q_value_to_error_rate(q_value)
    accuracy = 1 - error_rate
    likelihood *= accuracy
end

In [812]:
likelihood

0.0014241153255864945

In [814]:
for index in 2:length(original_oriented_path)
    kmer, orientation = original_oriented_path[index]
    prior_kmer, prior_orientation = original_oriented_path[index-1]
    state_likelihood = kmer_likelihoods[kmer]
    transition_likelihood = transition_likelihoods[prior_orientation][prior_kmer, kmer]
    sequence_index = kmer_graph.gprops[:k] + index - 1
    state_q_value = FASTX.quality(fastq_record)[sequence_index]
    state_error_likelihood = q_value_to_error_rate(state_q_value)
    emission_likelihood = 1 - state_error_likelihood
    
    likelihood *= state_likelihood * transition_likelihood * emission_likelihood
end

In [827]:
likelihood_increase = NumericIO.formatted(maximum_likelihood_path_value / likelihood, ndigits=1, charset=:ASCII)

"5E10"

In [828]:
println("new path is $(likelihood_increase) more likely than observed_sequence")

new path is 5E10 more likely than observed_sequence


In [ ]:
# write sequence to output fastq

In [ ]:
# for k in [11, 13, 17, 23, 31, 41, 53]
#     @show k
#     outgraph = "$(DIR)/simplified-kayvirus.k_$(k).all_ncbi.gfa"
#     if !isfile(outgraph)
#         accessions = ncbi_staph_phage_metadata[!, "Accession"]
#         fastx_iterator = (Mycelia.get_sequence(db="nuccore", accession=accession) for accession in accessions)
#         KMER_TYPE = BioSequences.BigDNAMer{k}
#         @time kmer_graph = Mycelia.fastx_to_kmer_graph(KMER_TYPE, fastx_iterator)
#         @time simplified_graph = Mycelia.simplify_kmer_graph(kmer_graph)
# #         ^ > 1 day down to 34 seconds!
#         Mycelia.graph_to_gfa(simplified_graph, outgraph)
#     end
# end